In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly 
import plotly.graph_objects as go
import plotly.express as px
%matplotlib qt
pd.set_option('mode.chained_assignment', None)

# Load raw and save ready data

In [ ]:
# path = f'Experiments\MultiplyTemperature\Exp5(12.5)\Exp5_down.csv'

path=''
while (path == '') or (not os.path.isfile(path)):
    path = input(f"Input data path: ")
_path_list = (path).split('\\')
folder_path = '\\'.join(_path_list[:-1])
data_name = _path_list[-1]
print(f'    Folder:\n{folder_path}\n    Data name:\n {data_name}')

In [ ]:
path

In [ ]:
df1 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up1.csv',index_col=0)
# df2 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up2.csv',index_col=0)
# df3 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up3.csv',index_col=0)
# df4 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up4.csv',index_col=0)


In [ ]:
df1

In [ ]:
rules = {
    1:'image_sweep_check',
    2: 'combine_check',
    # 'inner_processor_check':'OK_inner'
    0:'OK'
}
col_rules = {
    'Viscosity_mark': 'Viscosity_verbose',
    'Temperature_mark': 'Temperature_verbose',
}

df1 = df1.rename(columns=col_rules)
df1[['Viscosity_verbose','Temperature_verbose']] =\
    df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

# df2 = df2.rename(columns=col_rules)
# df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
# df2['time'] =df2['time']+ df1['time'].max()+1

# df3 = df3.rename(columns=col_rules)
# df3[['Viscosity_verbose','Temperature_verbose']] = df3[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df3['time'] =df3['time']+ df2['time'].max()+1

# df4 = df4.rename(columns=col_rules)
# df4[['Viscosity_verbose','Temperature_verbose']] = df4[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df4['time'] =df4['time']+ df3['time'].max()+1
# df = pd.concat([df1,df2,df3,df4])

df = df1.dropna(subset=['Viscosity', 'Temperature'])
df

In [ ]:
np.any(df['time'].duplicated())

In [ ]:
assert not os.path.isfile(f'{folder_path}\AData_{data_name}'), 'Data already exists'
df.to_csv(f'{folder_path}\AData_{data_name}',index=0)

# Load

In [2]:
path=''
while (path == '') or (not os.path.isfile(path)):
    path= input(f"Input data path: ")
_path_list = (path).split('\\')
folder_path = '\\'.join(_path_list[:-1])
data_name = _path_list[-1]
df = pd.read_csv(path)
df

,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
0,0.0,222.6,OK,15.1,OK_inner
1,1.0,222.6,OK,15.4,image_sweep_check
2,2.0,222.3,OK,15.4,image_sweep_check
3,3.0,221.4,OK,15.0,OK_inner
4,4.0,219.9,OK_inner,15.5,OK_inner
...,...,...,...,...,...
4552,4564.0,35.5,OK_inner,40.2,OK
4553,4565.0,35.6,OK_inner,40.2,OK
4554,4566.0,35.6,OK_inner,40.2,OK
4555,4567.0,35.5,combine_check,40.2,OK


# Process

In [8]:
temperature_cond = ((12 < df['Temperature']) & (df['Temperature'] < 45))
viscosity_cond = ((0 < df['Viscosity']) & (df['Viscosity'] < 400))

dfs = df[temperature_cond & viscosity_cond]
# dfs =df

In [9]:
# plot_rules = {'processor_sweep':2, 'OK':0, 'error':4, 'value_combine':3,
#        'inner_processor_check':1}
# df[['Viscosity_verbose','Temperature_verbose']] = df[['Viscosity_verbose','Temperature_verbose']].replace(plot_rules)
# dfs['time'] = dfs['time'] / 60


In [10]:
# T_goupT= dfs.groupby(by='Temperature')['Temperature']
# T_groupV=dfs.groupby(by='Temperature')['Viscosity']
# def foo(g):
#     whisker_width=0
#     q1 = g.quantile(0.45)                
#     q3 = g.quantile(0.55)
#     iqr = q3 - q1
#     return (g >= q1 - whisker_width*iqr) & (g <= q3 + whisker_width*iqr)
#     # return np.abs((g - g.median()) / iqr) < 2.22
#     # return np.abs((g - g.mean()) / g.std(ddof=0)) < 1

# error_mask= T_groupV.apply(foo ).droplevel([0]).reset_index(drop=True).to_numpy()

# dfs = dfs[error_mask]


# # T_goupT2= dfs2.groupby(by='Temperature')['Temperature']
# # T_groupV2=dfs2.groupby(by='Temperature')['Viscosity']
# # dfs2
# dfs

In [11]:
dfs_median = dfs[['Temperature','Viscosity']].groupby(by='Temperature').median()
dfs_mean = dfs[['Temperature','Viscosity']].groupby(by='Temperature').mean()

## Time plots

In [12]:
fig,ax_v =plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(dfs['time'],dfs['Viscosity'],color='red',marker='.')
ax_v.set_ylabel('Viscosity',color='red')

ax_T.scatter(dfs['time'],dfs['Temperature'],color='blue',marker='.')
ax_T.set_ylabel('Temperature',color='blue')

Text(0, 0.5, 'Temperature')

In [ ]:
dfs['Temperature_verbose'].unique()

In [15]:
colors_dict = {
    'OK': '#636efa',
    'OK_inner':'#636eff' ,
    'image_sweep_check': '#00cc96',
    'combine_check':'#000000'
}
# '#EF553B' - red
fig = go.Figure()
fig.add_traces([
    go.Scatter(
        x=dfs['time'],
        y=dfs['Viscosity'],
        name='Viscosity',
        mode='markers',
        marker=dict(size=5,
                    color=dfs['Viscosity_verbose'].replace(colors_dict))),
    go.Scatter(
        x=dfs['time'],
        y=dfs['Temperature'],
        name='Temperature',
        yaxis='y2',
        mode='markers',
        marker=dict(size=5,
                    symbol='square',
                    color=dfs['Temperature_verbose'].replace(colors_dict)),
    ),
])

fig.update_layout(
    hovermode="x unified",
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    yaxis2=dict(overlaying='y', side='right'),
)
fig.show()

## Temterature plots

In [18]:
fig = go.Figure()
fig = px.scatter(
    dfs,
    x="Temperature",
    y="Viscosity",
    color='Viscosity_verbose',
)

fig.add_traces([
    go.Scatter(name='Median',
               x=dfs_median.index,
               y=dfs_median['Viscosity'],
               mode='lines'),
    go.Scatter(name='Median',
               x=dfs_mean.index,
               y=dfs_mean['Viscosity'],
               mode='lines'),
])
fig.update_layout(
    # xaxis=dict(range=(12, 40.5)),
                #   yaxis=dict(range=(0, 100)),
                  margin=dict(l=0, r=0, b=0, t=0, pad=0))
fig.show()

# Saving regression

In [19]:
dfp = dfs_mean

In [22]:
assert not os.path.isfile(f'{folder_path}\ARegression_{data_name}'), 'Data already exists'
dfp.to_csv(f'{folder_path}\ARegression_{data_name}',index=0)